In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json
import random
import time
import pandas as pd
import numpy as np
import re
from datetime import datetime

def get_next_element(lst, current):
    try:
        index = lst.index(current)
        return lst[index + 1]
    except (ValueError, IndexError):
        return None

def clean_text(text): #Очистка текста от лишних пробелов и переносов строк. Заменяем все виды пробельных символов на один пробел
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_text(soup): #Извлечение и очистка всего текста со страницы. Извлекаем текст из тела страницы
    body = soup.body
    if not body:
        return ""
    texts = [clean_text(text) for text in body.stripped_strings] # Получаем все строки текста, очищенные от лишних пробелов
    cleaned_text = [text for text in texts if text] # Объединяем их в список, исключая пустые строки
    return cleaned_text

def extract_links(soup, base_url): #Извлечение всех гиперссылок со страницы
    links = []
    for link in soup.find_all("a", href=True):
        link_text = clean_text(link.get_text())
        href = urljoin(base_url, link['href'])  # Преобразуем относительные ссылки в абсолютные
        links.append({"text": link_text, "url": href})
    return links

def extract_elements(soup): #Извлечение всех элементов со страницы
    elements = []
    for tag in soup.find_all(True):  # Перебираем все теги
        tag_text = clean_text(tag.get_text())
        if tag_text:  # Добавляем только если есть текст
            elements.append({"tag": tag.name,
                             "attributes": tag.attrs,
                             "text": tag_text})
    return elements

In [6]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}

# Ваши исходные данные
publishdate = '26.12.2024' #datetime.now().strftime('%d.%m.%Y')
cnt_records = 50
keywords = ['лизинг','финансовая+аренда']

# Инициализация DataFrame
hdf = pd.DataFrame(columns=['№', 'link'])
filtered_data = []

for fz in [44,223]:
    for keyword in keywords:
        for page in range(1, 1000):
            # Задержка на случайное количество секунд
            sleep_time = random.randint(3, 7)
            print(f"Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
            
            # Формирование URL
            if fz == 44:
                url = (f"https://zakupki.gov.ru/epz/orderplan/search/results.html?"
                       f"searchString={keyword}&morphology=on&structuredCheckBox=on&structured=true&"
                       f"notStructured=on&fz44=on&planStatusTypes_0=on&planStatusTypes=0&"
                       f"publishDateFrom={publishdate}&sortBy=BY_MODIFY_DATE&pageNumber={page}&"
                       f"sortDirection=false&recordsPerPage=_{cnt_records}&showLotsInfoHidden=on&searchType=false")
            if fz == 223:
                url = (f"https://zakupki.gov.ru/epz/orderplan/search/results.html?"
               f"searchString={keyword}&morphology=on&structuredCheckBox=on&structured=true&"
               f"notStructuredCheckBox=on&notStructured=true&fz223=on&publishDateFrom={publishdate}&"
               f"sortBy=BY_MODIFY_DATE&pageNumber={page}&sortDirection=false&recordsPerPage={cnt_records}&showLotsInfoHidden=on&searchType=false")

            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()  # Проверка на успешный запрос
            except requests.RequestException as e:
                print(f"Ошибка при запросе к {url}: {e}")
                break  # Прерываем цикл страницы при ошибке запроса
    
            # Парсинг HTML
            soup = BeautifulSoup(response.text, "html.parser")
            raw_text = soup.get_text(separator="\n")
            cleantext = "\n".join([line.strip() for line in raw_text.splitlines() if line.strip()])
            data = {"url": url, "page_text": cleantext, "links": []}
            
            # Извлечение ссылок
            links = soup.find_all("a", href=True)
            for link in links:
                href = link['href'].strip()
                text = link.get_text(strip=True)
                full_url = urljoin(url, href)
                data["links"].append({"text": text, "href": full_url})
            
            # Сохранение данных в JSON (можно изменить режим на 'a' для добавления)
            with open("page_data.json", "w", encoding="utf-8") as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
            
            # Проверка наличия '№ ' в ссылках
            contains_num = False
            for link in data['links']:
                if link['text'][:2] == '№ ':
                    filtered_data.append({'№': link['text'], 'link': link['href']})
                    contains_num = True
            
            if contains_num:
                # Создание DataFrame из отфильтрованных данных
                df = pd.DataFrame(filtered_data, columns=['№', 'link'])
                df['fz'] = fz
                # Объединение с основным DataFrame
                hdf = pd.concat([hdf, df], ignore_index=True)
                # Очистка списка для следующей итерации
                filtered_data = []
            else:
                print(f"Страница {page} не содержит '№ ', прерываем цикл страниц для ключевого слова '{keyword}'.")
                break  # Прерываем цикл страниц, если '№ ' не найдено
            
            print(f"Ключевое слово: {keyword}, Страница: {page}")
    
        print(f"Завершен обработка ключевого слова: {keyword}")

Sleeping for 3 seconds...
Ключевое слово: лизинг, Страница: 1
Sleeping for 7 seconds...
Страница 2 не содержит '№ ', прерываем цикл страниц для ключевого слова 'лизинг'.
Завершен обработка ключевого слова: лизинг
Sleeping for 4 seconds...
Ключевое слово: финансовая+аренда, Страница: 1
Sleeping for 4 seconds...
Страница 2 не содержит '№ ', прерываем цикл страниц для ключевого слова 'финансовая+аренда'.
Завершен обработка ключевого слова: финансовая+аренда
Sleeping for 4 seconds...
Ключевое слово: лизинг, Страница: 1
Sleeping for 3 seconds...
Страница 2 не содержит '№ ', прерываем цикл страниц для ключевого слова 'лизинг'.
Завершен обработка ключевого слова: лизинг
Sleeping for 6 seconds...
Ключевое слово: финансовая+аренда, Страница: 1
Sleeping for 7 seconds...
Страница 2 не содержит '№ ', прерываем цикл страниц для ключевого слова 'финансовая+аренда'.
Завершен обработка ключевого слова: финансовая+аренда


In [7]:
n = hdf.drop_duplicates().reset_index(drop=True)
n['fz'] = n['fz'].astype(int)
n[['customer_inn','customer_kpp','name','publish_date','plan_year','start_date','phone','fio','okopf','link_attach','link_doc']] = None,None,None,None,None,None,None,None,None,None,None

In [8]:
for i in n.index:
    if n.loc[n.index==i]['fz'].values[0] == 44:
        headers = {"User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) ""AppleWebKit/537.36 (KHTML, like Gecko) ""Chrome/112.0.0.0 Safari/537.36")}
        url = n.loc[n.index==i]['link'].values[0]
        response = requests.get(url, headers=headers)
        try:
            sleep_time = random.randint(2, 4)
            time.sleep(sleep_time)
            response = requests.get(url, headers=headers, timeout=120)
            response.raise_for_status()  # Проверяет статус и вызывает исключение для ошибок HTTP
        except requests.RequestException as e:
            print(f"Ошибка при запросе к {url}: {e}")
        
        soup = BeautifulSoup(response.text, "html.parser")
        clean = clean_text(soup.title.string) if soup.title else "Без заголовка"
        extract = extract_text(soup)
        links = extract_links(soup, url)
        elements = extract_elements(soup)
        
        structured_data = {"url": url,
                           "title": clean,
                           "full_text": extract,
                           "links": links,
                           "elements": elements}
        
        data = json.loads(json.dumps(structured_data, ensure_ascii=False, indent=4))
        n.loc[n.index==i,'customer_inn'] = get_next_element(data['full_text'], 'ИНН/КПП').split('/')[0]
        n.loc[n.index==i,'customer_kpp'] = get_next_element(data['full_text'], 'ИНН/КПП').split('/')[1]
        n.loc[n.index==i,'name'] = get_next_element(data['full_text'], 'Заказчик')
        n.loc[n.index==i,'publish_date'] = get_next_element(data['full_text'], 'Дата размещения плана-графика закупок')
        n.loc[n.index==i,'plan_year'] = get_next_element(data['full_text'], 'Финансовый год планирования')
        n.loc[n.index==i,'start_date'] = get_next_element(data['full_text'], 'Плановый период')
        n.loc[n.index==i,'phone'] = get_next_element(data['full_text'], 'Телефон')
        n.loc[n.index==i,'fio'] = get_next_element(data['full_text'], 'ФИО лица, утвердившего план-график закупок')
        n.loc[n.index==i,'okopf'] = get_next_element(data['full_text'], 'ОКОПФ')
        for l in data['links']:
            if l['text'] == 'Вложения':
                n.loc[n.index==i,'link_attach'] = l['url']

    if n.loc[n.index==i]['fz'].values[0] == 223:
        headers = {"User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) ""AppleWebKit/537.36 (KHTML, like Gecko) ""Chrome/112.0.0.0 Safari/537.36")}
        url = n.loc[n.index==i]['link'].values[0]
        response = requests.get(url, headers=headers)
        try:
            sleep_time = random.randint(1, 4)
            time.sleep(sleep_time)
            response = requests.get(url, headers=headers, timeout=120)
            response.raise_for_status()  # Проверяет статус и вызывает исключение для ошибок HTTP
        except requests.RequestException as e:
            print(f"Ошибка при запросе к {url}: {e}")
        
        soup = BeautifulSoup(response.text, "html.parser")
        clean = clean_text(soup.title.string) if soup.title else "Без заголовка"
        extract = extract_text(soup)
        links = extract_links(soup, url)
        elements = extract_elements(soup)
        
        structured_data = {"url": url,
                           "title": clean,
                           "full_text": extract,
                           "links": links,
                           "elements": elements}
        
        data = json.loads(json.dumps(structured_data, ensure_ascii=False, indent=4))
        n.loc[n.index==i,'customer_inn'] = get_next_element(data['full_text'], 'ИНН')
        n.loc[n.index==i,'customer_kpp'] = get_next_element(data['full_text'], 'КПП')
        n.loc[n.index==i,'name'] = get_next_element(data['full_text'], 'Заказчик')
        n.loc[n.index==i,'publish_date'] = get_next_element(data['full_text'], 'Размещено')
        n.loc[n.index==i,'plan_year'] = get_next_element(data['full_text'], 'Период планирования')
        n.loc[n.index==i,'start_date'] = get_next_element(data['full_text'], 'Период действия плана')
        n.loc[n.index==i,'phone'] = get_next_element(data['full_text'], 'Телефон')
        n.loc[n.index==i,'fio'] = get_next_element(data['full_text'], 'ФИО лица, утвердившего план-график закупок')
        n.loc[n.index==i,'okopf'] = get_next_element(data['full_text'], 'ОКОПФ')

        for l in data['links']:
            if l['text'] == 'Документы':
                n.loc[n.index==i,'link_attach'] = l['url']

In [9]:
n

,№,link,fz,customer_inn,customer_kpp,name,publish_date,plan_year,start_date,phone,fio,okopf,link_attach,link_doc
0,№ 202505426000018001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,6369012303,636901001,"МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ""КОМХОЗ"" М...",26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-84664-33773,ГАРАЕВ Р. Ф.,65243: Муниципальное унитарное предприятие,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,None
1,№ 202408146000020001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,0600013347,060001001,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ЖИЛИЩНО-КОМ...",26.12.2024,2024 (2025 – 2026 года),2025 - 2026,7-8732-222310,ЭКАЖЕВ Х. М.,75404: Муниципальное казенное учреждение,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,None
2,№ 202503481000851001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,5015012008,501501001,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,26.12.2024,2025 (2026 – 2027 года),2026 - 2027,74959924213,Миронов М. Г.,75103: Федеральное государственное бюджетное у...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,None
3,№ 202508482000005001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,5024130840,502401001,ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ МОСКОВСКОЙ...,26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-498-6028344,Дупленская Е. А.,75204: Государственное казенное учреждение суб...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,None
4,№ 202503623003480001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,6640002543,668001001,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""СЛУЖБА ЗАКА...",26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-34389-22750,СИДОРОВ В. Н.,75404: Муниципальное казенное учреждение,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,None
5,№ 202503621000100001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,6659014366,667801001,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,26.12.2024,2025 (2026 – 2027 года),2026 - 2027,73432212415,Азаров Е. Б.,75103: Федеральное государственное бюджетное у...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,None
6,№ 202503871000045001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,8601043081,860101001,СЛЕДСТВЕННОЕ УПРАВЛЕНИЕ СЛЕДСТВЕННОГО КОМИТЕТА...,26.12.2024,2025 (2026 – 2027 года),2026 - 2027,6-43346-7328558,Мязин А. А.,75104: Федеральное государственное казенное уч...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,None
7,№ 202503522000236001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,5506017015,550301001,"КАЗЕННОЕ УЧРЕЖДЕНИЕ ОМСКОЙ ОБЛАСТИ ""РЕГИОНАЛЬН...",26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-3812-581737,Гитал Т. В.,75204: Государственное казенное учреждение суб...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,None
8,№ 202503382000136001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,4101171828,410101001,"КРАЕВОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""Ц...",26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-4152-200099,Гончарова Е. Б.,75204: Государственное казенное учреждение суб...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,None
9,№ 2240800170,https://zakupki.gov.ru/epz/orderplan/purchase-...,223,6608002436,665801001,ЕКАТЕРИНБУРГСКОЕ МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДП...,26.12.2024,2025,с 01.01.2025 по 31.12.2025,None,None,None,https://zakupki.gov.ru/epz/orderplan/purchase-...,None


In [10]:
fz_dict = {44:'https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=',
          223:'https://zakupki.gov.ru/223/plan/public/plan-info/print-form/show-with-paging.html?planInfoId='}
for fz in [44,223]:
    for i in n[n['fz']==fz].index:
        headers = {"User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) ""AppleWebKit/537.36 (KHTML, like Gecko) ""Chrome/112.0.0.0 Safari/537.36")}
        url = n.loc[n.index==i,'link_attach'].values[0]
        response = requests.get(url, headers=headers)
        try:
            sleep_time = random.randint(1, 3)
            time.sleep(sleep_time)
            response = requests.get(url, headers=headers, timeout=120)
            response.raise_for_status()  # Проверяет статус и вызывает исключение для ошибок HTTP
        except requests.RequestException as e:
            print(f"Ошибка при запросе к {url}: {e}")
        soup = BeautifulSoup(response.text, "html.parser")
        clean = clean_text(soup.title.string) if soup.title else "Без заголовка"
        extract = extract_text(soup)
        links = extract_links(soup, url)
        elements = extract_elements(soup)
        structured_data = {"url": url,"title": clean,"full_text": extract,"links": links,"elements": elements}
        data = json.loads(json.dumps(structured_data, ensure_ascii=False, indent=4))
        for k in data['links']:
            if (k['text'] == '') and (fz_dict[fz] in k['url']):
                n.loc[n.index==i,'link_doc'] = k['url']
                break
    

In [13]:
1

1

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Функция для получения содержимого страницы
def get_page_content(url, headers):
    # Список интервалов ожидания в секундах (1 мин, 2 мин, 3 мин, 5 мин, 10, 15, 20, 25, 30 мин)
    wait_times = [45, 1114, 177, 258, 544, 901, 1208, 1444, 1782]
    attempt = 0  # Номер текущей попытки

    while attempt <= len(wait_times):
        try:
            # Случайная задержка перед запросом (2-4 секунды)
            sleep_time = random.randint(2, 4)
            time.sleep(sleep_time)
            
            # Выполнение GET-запроса
            response = requests.get(url, headers=headers)
            
            # Проверка статуса ответа
            if response.status_code == 200:
                response.encoding = response.apparent_encoding
                return response.text
            else:
                print(f"Ошибка запроса {response.status_code} для URL: {url}")
        
        except requests.exceptions.RequestException as e:
            # Обработка исключений, связанных с запросом
            print(f"Исключение при запросе к {url}: {e}")
        
        # Проверка, есть ли еще попытки
        if attempt < len(wait_times):
            wait = wait_times[attempt]
            minutes = wait // 60
            print(f"Повторная попытка через {minutes} минут.")
            time.sleep(wait)  # Ожидание перед следующей попыткой
            attempt += 1
        else:
            print("Максимальное количество попыток достигнуто. Запрос не удался.")
            return None

df_pos = pd.DataFrame(columns = ['position_number','okpd2','okpd2_names','purchase_object','summ','data_notif','pos_start_date','type','link_doc'])

for fz in [44,223]:
    for i in n[n['fz']==fz].index:
        base_url = n.loc[n.index==i]['link_doc'].values[0].split('&page')[0]+'&page={}' # Замените на реальный URL с пагинацией
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}
        all_rows = []
        previous_page_content = None
        page_number = 1
        while True:
            # Формируем URL для текущей страницы
            url = base_url.format(page_number)
            print(f"Парсинг страницы {page_number}: {url}")
            
            # Получаем содержимое страницы
            page_content = get_page_content(url, headers)
            if page_content is None:
                break  # Если произошла ошибка запроса, выходим из цикла
            
            # Сравниваем текущую страницу с предыдущей
            if page_content == previous_page_content:
                print("Повтор страницы. Останавливаем парсинг.")
                break
            
            # Парсим содержимое текущей страницы
            soup = BeautifulSoup(page_content, "lxml")
            all_tables = soup.find_all("table")
            print(f"Найдено таблиц на странице {page_number}: {len(all_tables)}")
            
            # Если таблиц нет, пропускаем страницу
            if not all_tables:
                print(f"Таблицы не найдены на странице {page_number}. Пропускаем.")
                page_number += 1
                continue
            if page_number>1:
                df2 = df1.drop_duplicates().reset_index(drop=True).copy()
            
            # Обрабатываем таблицы
            for idx, table in enumerate(all_tables):
                rows = table.find_all("tr")
                for row in rows:
                    cells = row.find_all(["td", "th"])
                    row_text = [cell.get_text(strip=True) for cell in cells]
                    # Добавляем идентификатор страницы и таблицы
                    row_text.append(f"Page_X")
                    #row_text.append(f"Page_{page_number}")
                    row_text.append(f"Table_{idx + 1}")
                    all_rows.append(row_text)

            max_columns = max(len(row) for row in all_rows)
            df1 = pd.DataFrame(all_rows, columns=[f"Column_{i}" for i in range(max_columns - 2)] + ["Page_ID", "Table_ID"])
            df1 = df1.drop_duplicates().reset_index(drop=True)
            if page_number>1:
                if df1.shape == df2.shape:
                    break
            # Обновляем содержимое предыдущей страницы
            previous_page_content = page_content
            page_number += 1
        
        # Проверяем, есть ли собранные строки
        if not all_rows:
            print("Нет данных для создания DataFrame.")
        else:
            # Преобразуем в DataFrame
            max_columns = max(len(row) for row in all_rows)
            columns = [f"Column_{i}" for i in range(max_columns - 2)] + ["Page_ID", "Table_ID"]
            df = pd.DataFrame(all_rows, columns=columns)
            if fz == 44:
                df = df[(df['Column_5'].astype(str).apply(len)==4)&(df['Column_2'].astype(str).apply(len)>1)].dropna()
                df['Column_6'] = pd.to_numeric(df['Column_6'], errors="coerce").fillna(0)
                df = df[df['Column_6']>500000]
                df = df[['Column_1','Column_2','Column_3','Column_4','Column_6','Column_5']].drop_duplicates()
                df['pos_start_date'] = None
                df['type'] = None
                df['link_doc'] = n.loc[n.index==i]['link_doc'].values[0]
                df.columns = ['position_number','okpd2','okpd2_names','purchase_object','summ','data_notif','pos_start_date','type','link_doc']
            if fz == 223:
                df = df[df['Column_14'].isin(['Да','Нет'])]
                df = df[['Column_0','Column_2','Column_1','Column_3','Column_10','Column_11','Column_12','Column_13']]
                df['link_doc'] = n.loc[n.index==i]['link_doc'].values[0]
                df.columns = ['position_number','okpd2','okpd2_names','purchase_object','summ','data_notif','pos_start_date','type','link_doc']
            df_pos = pd.concat([df,df_pos], ignore_index=True)

Парсинг страницы 1: https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=76606667&source=pg2020PF&page=1
Найдено таблиц на странице 1: 4
Парсинг страницы 2: https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=76606667&source=pg2020PF&page=2
Повтор страницы. Останавливаем парсинг.
Парсинг страницы 1: https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=76462219&source=pg2020PF&page=1


C:\Users\79259\AppData\Local\Temp\ipykernel_22680\1354300740.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_pos = pd.concat([df,df_pos], ignore_index=True)


Найдено таблиц на странице 1: 4
Парсинг страницы 2: https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=76462219&source=pg2020PF&page=2
Повтор страницы. Останавливаем парсинг.
Парсинг страницы 1: https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=76567663&source=pg2020PF&page=1
Найдено таблиц на странице 1: 4
Парсинг страницы 2: https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=76567663&source=pg2020PF&page=2
Повтор страницы. Останавливаем парсинг.
Парсинг страницы 1: https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=76530764&source=pg2020PF&page=1
Найдено таблиц на странице 1: 4
Парсинг страницы 2: https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=76530764&source=pg2020PF&page=2
Повтор страницы. Останавливаем парсинг.
Парсинг страницы 1: https://zakupki.gov.ru/epz/orderplan/printForm/view.html?printFormId=76521045&source=pg2020PF&page=1
Найдено таблиц на странице 1: 4
Парсинг страницы 2: https:

In [16]:
t1 = n.merge(df_pos,on='link_doc',how='left')

In [17]:
t1 = t1[t1['okpd2'].astype(str).str.startswith('77')|t1['okpd2'].astype(str).str.startswith('64')].reset_index(drop=True)

In [18]:
t1

,№,link,fz,customer_inn,customer_kpp,name,publish_date,plan_year,start_date,phone,...,link_attach,link_doc,position_number,okpd2,okpd2_names,purchase_object,summ,data_notif,pos_start_date,type
0,№ 202505426000018001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,6369012303,636901001,"МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ""КОМХОЗ"" М...",26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-84664-33773,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,253636901230363690100100030007712000,77.12.19.000,Услуги по аренде и лизингу прочих сухопутных т...,Услуги по аренде и лизингу прочих сухопутных т...,1602555.7,2025,None,None
1,№ 202408146000020001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,0600013347,060001001,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ЖИЛИЩНО-КОМ...",26.12.2024,2024 (2025 – 2026 года),2025 - 2026,7-8732-222310,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,243060001334706000100100010007739244,77.39.19.113,Услуги по аренде и лизингу подъемно-транспортн...,Услуги по аренде и лизингу подъемно-транспортн...,10020000.0,2024,None,None
2,№ 202508482000005001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,5024130840,502401001,ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ МОСКОВСКОЙ...,26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-498-6028344,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,262502413084050240100100840007739242,77.39,"Услуги по аренде и лизингу прочих машин, обору...",Оказание услуг по предоставлению стоечного про...,900000000.0,2026,None,None
3,№ 202508482000005001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,5024130840,502401001,ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ МОСКОВСКОЙ...,26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-498-6028344,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,272502413084050240100101290007739242,77.39,"Услуги по аренде и лизингу прочих машин, обору...",Оказание услуг по предоставлению стоечного про...,900000000.0,2027,None,None
4,№ 202503623003480001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,6640002543,668001001,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""СЛУЖБА ЗАКА...",26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-34389-22750,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,253664000254366800100100320007739244,77.39.19.129,Услуги по аренде и лизингу прочих материальных...,Ограниченное пользование воздушных линий элект...,11113186.4,2025,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,№ 2240799208,https://zakupki.gov.ru/epz/orderplan/purchase-...,223,7706433263,770601001,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,26.12.2024,2025,с 01.01.2025 по 31.12.2025,None,...,https://zakupki.gov.ru/epz/orderplan/purchase-...,https://zakupki.gov.ru/223/plan/public/plan-in...,10,77.39.19.112,77.39.22,Оказания услуг финансовой аренды (лизинга) бур...,54 059 658.04 Российский рубльВ том числе объе...,08.2023,11.2026,Запрос котировок
80,№ 2240799208,https://zakupki.gov.ru/epz/orderplan/purchase-...,223,7706433263,770601001,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,26.12.2024,2025,с 01.01.2025 по 31.12.2025,None,...,https://zakupki.gov.ru/epz/orderplan/purchase-...,https://zakupki.gov.ru/223/plan/public/plan-in...,105,77.39.19.115,77.39.25,Оказания услуг финансовой аренды (лизинга) ком...,36 858 108.83 Российский рубльВ том числе объе...,11.2022,03.2025,Конкурс в электронной форме
81,№ 2240796916,https://zakupki.gov.ru/epz/orderplan/purchase-...,223,8606013108,860601001,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВОДОКАНАЛ""",26.12.2024,с 2025 по 2027,с 01.01.2025 по 31.12.2027,None,...,https://zakupki.gov.ru/epz/orderplan/purchase-...,https://zakupki.gov.ru/223/plan/public/plan-in...,3,64.91,64.91,Оказание услуг финансовой аренды (лизинга) ав...,7 636 840.8

In [19]:
t1

,№,link,fz,customer_inn,customer_kpp,name,publish_date,plan_year,start_date,phone,...,link_attach,link_doc,position_number,okpd2,okpd2_names,purchase_object,summ,data_notif,pos_start_date,type
0,№ 202505426000018001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,6369012303,636901001,"МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ""КОМХОЗ"" М...",26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-84664-33773,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,253636901230363690100100030007712000,77.12.19.000,Услуги по аренде и лизингу прочих сухопутных т...,Услуги по аренде и лизингу прочих сухопутных т...,1602555.7,2025,None,None
1,№ 202408146000020001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,0600013347,060001001,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ЖИЛИЩНО-КОМ...",26.12.2024,2024 (2025 – 2026 года),2025 - 2026,7-8732-222310,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,243060001334706000100100010007739244,77.39.19.113,Услуги по аренде и лизингу подъемно-транспортн...,Услуги по аренде и лизингу подъемно-транспортн...,10020000.0,2024,None,None
2,№ 202508482000005001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,5024130840,502401001,ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ МОСКОВСКОЙ...,26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-498-6028344,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,262502413084050240100100840007739242,77.39,"Услуги по аренде и лизингу прочих машин, обору...",Оказание услуг по предоставлению стоечного про...,900000000.0,2026,None,None
3,№ 202508482000005001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,5024130840,502401001,ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ МОСКОВСКОЙ...,26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-498-6028344,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,272502413084050240100101290007739242,77.39,"Услуги по аренде и лизингу прочих машин, обору...",Оказание услуг по предоставлению стоечного про...,900000000.0,2027,None,None
4,№ 202503623003480001,https://zakupki.gov.ru/epz/orderplan/pg2020/ge...,44,6640002543,668001001,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""СЛУЖБА ЗАКА...",26.12.2024,2025 (2026 – 2027 года),2026 - 2027,7-34389-22750,...,https://zakupki.gov.ru/epz/orderplan/pg2020/do...,https://zakupki.gov.ru/epz/orderplan/printForm...,253664000254366800100100320007739244,77.39.19.129,Услуги по аренде и лизингу прочих материальных...,Ограниченное пользование воздушных линий элект...,11113186.4,2025,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,№ 2240799208,https://zakupki.gov.ru/epz/orderplan/purchase-...,223,7706433263,770601001,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,26.12.2024,2025,с 01.01.2025 по 31.12.2025,None,...,https://zakupki.gov.ru/epz/orderplan/purchase-...,https://zakupki.gov.ru/223/plan/public/plan-in...,10,77.39.19.112,77.39.22,Оказания услуг финансовой аренды (лизинга) бур...,54 059 658.04 Российский рубльВ том числе объе...,08.2023,11.2026,Запрос котировок
80,№ 2240799208,https://zakupki.gov.ru/epz/orderplan/purchase-...,223,7706433263,770601001,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,26.12.2024,2025,с 01.01.2025 по 31.12.2025,None,...,https://zakupki.gov.ru/epz/orderplan/purchase-...,https://zakupki.gov.ru/223/plan/public/plan-in...,105,77.39.19.115,77.39.25,Оказания услуг финансовой аренды (лизинга) ком...,36 858 108.83 Российский рубльВ том числе объе...,11.2022,03.2025,Конкурс в электронной форме
81,№ 2240796916,https://zakupki.gov.ru/epz/orderplan/purchase-...,223,8606013108,860601001,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВОДОКАНАЛ""",26.12.2024,с 2025 по 2027,с 01.01.2025 по 31.12.2027,None,...,https://zakupki.gov.ru/epz/orderplan/purchase-...,https://zakupki.gov.ru/223/plan/public/plan-in...,3,64.91,64.91,Оказание услуг финансовой аренды (лизинга) ав...,7 636 840.8

In [ ]:
df_pos.to_excel('df_pos.xlsx')

In [ ]:
str(df_pos[1467:1468]['summ'].values[0])

In [ ]:
t1 = n.merge(df_pos,on='link_doc',how='left')

In [ ]:
t1.shape

In [ ]:
t1 = t1[t1['okpd2'].astype(str).str.startswith('77')|t1['okpd2'].astype(str).str.startswith('64')].reset_index(drop=True)

In [ ]:
t1.to_excel('t1.xlsx')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Функция для получения содержимого страницы
def get_page_content(url, headers):
    sleep_time = random.randint(2, 4)
    time.sleep(sleep_time)
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Ошибка запроса {response.status_code} для URL: {url}")
        return None
    response.encoding = response.apparent_encoding
    return response.text

# Указанные параметры
#doc_url = n['link_doc'].values[0].split('&page')[0]+'&page={}"
base_url = n['link_doc'].values[-1].split('&page')[0]+'&page={}' # Замените на реальный URL с пагинацией
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
}

# Список для хранения всех строк всех таблиц
all_rows = []

# Переменные для трекинга страниц
previous_page_content = None
page_number = 1

while True:
    # Формируем URL для текущей страницы
    url = base_url.format(page_number)
    print(f"Парсинг страницы {page_number}: {url}")
    
    # Получаем содержимое страницы
    page_content = get_page_content(url, headers)
    if page_content is None:
        break  # Если произошла ошибка запроса, выходим из цикла
    
    # Сравниваем текущую страницу с предыдущей
    if page_content == previous_page_content:
        print("Повтор страницы. Останавливаем парсинг.")
        break
    
    # Парсим содержимое текущей страницы
    soup = BeautifulSoup(page_content, "lxml")
    all_tables = soup.find_all("table")
    print(f"Найдено таблиц на странице {page_number}: {len(all_tables)}")
    
    # Если таблиц нет, пропускаем страницу
    if not all_tables:
        print(f"Таблицы не найдены на странице {page_number}. Пропускаем.")
        page_number += 1
        continue
    
    # Обрабатываем таблицы
    for idx, table in enumerate(all_tables):
        rows = table.find_all("tr")
        for row in rows:
            cells = row.find_all(["td", "th"])
            row_text = [cell.get_text(strip=True) for cell in cells]
            # Добавляем идентификатор страницы и таблицы
            row_text.append(f"Page_{page_number}")
            row_text.append(f"Table_{idx + 1}")
            all_rows.append(row_text)
    
    # Обновляем содержимое предыдущей страницы
    previous_page_content = page_content
    page_number += 1

# Проверяем, есть ли собранные строки
if not all_rows:
    print("Нет данных для создания DataFrame.")
else:
    # Преобразуем в DataFrame
    max_columns = max(len(row) for row in all_rows)
    columns = [f"Column_{i}" for i in range(max_columns - 2)] + ["Page_ID", "Table_ID"]
    df = pd.DataFrame(all_rows, columns=columns)
    df = df[df['Column_14'].isin(['Да','Нет'])]
    df = df[['Column_0','Column_2','Column_1','Column_3','Column_10','Column_11','Column_12','Column_13']]
    df['link_doc'] = n['link_doc'].values[-1]
    df.columns = ['position_number','okpd2','okpd2_names','purchase_object','summ','data_notif','pos_start_date','type','link_doc']

In [ ]:
df

In [ ]:
url = n['link_doc'].values[0]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

resp = requests.get(url)
resp.encoding = resp.apparent_encoding

soup = BeautifulSoup(resp.text, "lxml")

# Найдём все таблицы
all_tables = soup.find_all("table")
print("Всего таблиц:", len(all_tables))

# Допустим, большая таблица - это последняя (или нам видно, что это 4-я по счёту)
table = all_tables[-1]  # Или all_tables[3], если вручную определили индекс

# Берём thead (если есть) и tbody (если есть)
thead = table.find("thead")
tbody = table.find("tbody")

# Примерный алгоритм: собрать заголовки <th> или первую <tr> как имена столбцов
headers = []
if thead:
    # Иногда нужные заголовки лежат в нескольких tr. Или вы берёте просто первую строку
    header_rows = thead.find_all("tr")
    # В простейшем случае, берем последнюю строку thead (часто в ней реальные названия)
    last_header_row = header_rows[-1]
    for cell in last_header_row.find_all(["th", "td"]):
        headers.append(cell.get_text(strip=True))
else:
    # Если thead нет, берем первую строчку tbody в качестве заголовков
    first_row = table.find("tr")
    for cell in first_row.find_all(["th", "td"]):
        headers.append(cell.get_text(strip=True))

print("Заголовки:", headers)

# Далее строки данных
rows_data = []
for row in tbody.find_all("tr"):
    cells = row.find_all(["td", "th"])
    row_text = [c.get_text(strip=True) for c in cells]
    rows_data.append(row_text)

# Теперь можно собрать всё в pandas.DataFrame:
import pandas as pd
df = pd.DataFrame(rows_data, columns=headers)
df[df['3'].apply(len)>1].dropna()
print(df.head())

In [ ]:
all_tables

In [ ]:
df = df[df['3'].apply(len)>1].dropna()

In [ ]:
df

In [ ]:
import re

# Исходный список данных
data_list = data['full_text']

# Регулярные выражения
pattern_okpd2 = re.compile(r'\d+\.\d+:')
pattern_price = re.compile(r'.*,\d{2}$')

# Разбивка списка на блоки по 4 элемента
blocks = [data_list[i:i + 4] for i in range(0, len(data_list), 4)]

matched_blocks = []

for block in blocks:
    if len(block) == 4:
        code, purchase_item, okpd2, price = block
        if pattern_okpd2.search(okpd2) and pattern_price.search(price):
            matched_blocks.append({
                'Код закупки': code,
                'Предмет закупки': purchase_item,
                'ОКПД2': okpd2,
                'Цена': price
            })
    else:
        print(f"Неполный блок: {block}")

# Вывод результатов
for block in matched_blocks:
    print("Код закупки:", block['Код закупки'])
    print("Предмет закупки:", block['Предмет закупки'])
    print("ОКПД2:", block['ОКПД2'])
    print("Цена:", block['Цена'])
    print("---")

In [ ]:
matched_blocks

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# URL первой страницы
base_url = 'https://zakupki.gov.ru/epz/orderplan/pg2020/plan-position.html?plan-number=202503873000018001&revision-id=&position-number='  # Замените на нужный URL

# Заголовки для имитации браузера
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

# Таймаут в секундах
REQUEST_TIMEOUT = 10  # Можно настроить в зависимости от ваших потребностей

# Количество попыток при возникновении ошибок
MAX_RETRIES = 3

# Задержка между попытками (в секундах)
RETRY_DELAY = 5

# Функция для получения BeautifulSoup объекта с обработкой таймаута и ошибок
def get_soup(url):
    attempts = 0
    while attempts < MAX_RETRIES:
        try:
            response = requests.get(url, headers=headers, timeout=REQUEST_TIMEOUT)
            response.raise_for_status()  # Проверка на HTTP ошибки
            return BeautifulSoup(response.text, 'html.parser')
        except requests.exceptions.Timeout:
            print(f"Таймаут при запросе {url}. Попытка {attempts + 1} из {MAX_RETRIES}.")
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP ошибка при запросе {url}: {http_err}")
            break  # Не имеет смысла повторять запрос при HTTP ошибках
        except requests.exceptions.RequestException as err:
            print(f"Ошибка при запросе {url}: {err}")
            break
        attempts += 1
        time.sleep(RETRY_DELAY)
    print(f"Не удалось получить страницу {url} после {MAX_RETRIES} попыток.")
    return None

# Функция для поиска ссылки на вторую страницу
def find_second_page_url(soup, current_url):
    # Попытка найти ссылку с текстом "2"
    second_page_link = soup.find('a', text='2')
    if not second_page_link:
        # Альтернативный способ: поиск внутри контейнера пагинации
        pagination = soup.find('div', class_='pagination')
        if pagination:
            second_page_link = pagination.find('a', text='2')
    
    if second_page_link:
        href = second_page_link.get('href')
        # Проверка на относительную ссылку
        second_page_url = urljoin(current_url, href)
        return second_page_url
    return None

# Получаем первую страницу
soup = get_soup(base_url)
if soup is None:
    exit()

# Ищем ссылку на вторую страницу
second_page_url = find_second_page_url(soup, base_url)
if second_page_url:
    print(f"Ссылка на вторую страницу: {second_page_url}")
    
    # Получаем вторую страницу с обработкой таймаута
    soup_second = get_soup(second_page_url)
    if soup_second:
        print("Вторая страница успешно загружена.")
        # Пример обработки содержимого второй страницы
        # Например, вывод заголовков статей
        articles = soup_second.find_all('h2', class_='article-title')
        for idx, article in enumerate(articles, start=1):
            print(f"{idx}. {article.get_text(strip=True)}")
    else:
        print("Не удалось загрузить вторую страницу.")
else:
    print("Ссылка на вторую страницу не найдена.")

In [ ]:
for l in data['links']:
    if l['text'] == 'Позиции плана закупки':
        print(l)

In [ ]:
headers = {"User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) ""AppleWebKit/537.36 (KHTML, like Gecko) ""Chrome/112.0.0.0 Safari/537.36")}
url = n.loc[n.index==0]['link'].values[0]
response = requests.get(url, headers=headers)
try:
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()  # Проверяет статус и вызывает исключение для ошибок HTTP
except requests.RequestException as e:
    print(f"Ошибка при запросе к {url}: {e}")

soup = BeautifulSoup(response.text, "html.parser")
clean = clean_text(soup.title.string) if soup.title else "Без заголовка"
extract = extract_text(soup)
links = extract_links(soup, url)
elements = extract_elements(soup)

structured_data = {"url": url,
                   "title": clean,
                   "full_text": extract,
                   "links": links,
                   "elements": elements}

json_output = json.dumps(structured_data, ensure_ascii=False, indent=4)
with open("structured_data.json", "w", encoding="utf-8") as f: f.write(json_output)
with open("structured_data.json", "r", encoding="utf-8") as file: data = json.load(file)

In [ ]:
data['links']

In [ ]:
get_next_element(data['full_text'], 'ИНН')

In [ ]:
n.loc[n.index==i,'customer_inn'] = get_next_element(data['full_text'], 'ИНН').split('/')[0]
n.loc[n.index==i,'customer_kpp'] = get_next_element(data['full_text'], 'КПП').split('/')[1]
n.loc[n.index==i,'name'] = get_next_element(data['full_text'], 'Заказчик')
n.loc[n.index==i,'publish_date'] = get_next_element(data['full_text'], 'Размещено')
n.loc[n.index==i,'plan_year'] = get_next_element(data['full_text'], 'Период планирования')
n.loc[n.index==i,'start_date'] = get_next_element(data['full_text'], 'Период действия плана')
n.loc[n.index==i,'phone'] = get_next_element(data['full_text'], 'Телефон')
n.loc[n.index==i,'fio'] = get_next_element(data['full_text'], 'ФИО лица, утвердившего план-график закупок')
n.loc[n.index==i,'okopf'] = get_next_element(data['full_text'], 'ОКОПФ')
n.loc[n.index==i,'start_date'] = get_next_element(data['full_text'], 'Период действия плана')

In [ ]:
import json

# Открываем файл JSON
with open("page_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Пример работы с данными
print("URL страницы:", data["url"])
print("Текст страницы:")
print(data["page_text"])

print("\nСсылки:")
for link in data["links"]:
    print(f"Текст: {link['text']} | Ссылка: {link['href']}")

In [ ]:
import pandas as pd

filtered_data = []

for k in data['links']:
    if '№ ' in k['text']:
        filtered_data.append({'№': k['text'], 'link': k['href']})

df = pd.DataFrame(filtered_data, columns=['№', 'link'])
df = pd.DataFrame([{'№': k['text'], 'link': k['href']} for k in data['links'] if '№ ' in k['text']],columns=['№', 'link'])
all_links_df = pd.DataFrame(data['links'])
df = all_links_df[all_links_df['text'].str.contains('№ ')][['text', 'href']].rename(columns={'text': '№', 'href': 'link'})


df

In [ ]:
if '№ ' in k and 'purchase-plan' in str(v):
        